<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#潜在语义模型练习" data-toc-modified-id="潜在语义模型练习-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>潜在语义模型练习</a></span></li><li><span><a href="#潜在语义分析（LSA）" data-toc-modified-id="潜在语义分析（LSA）-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>潜在语义分析（LSA）</a></span><ul class="toc-item"><li><span><a href="#奇异值分解（SVD）" data-toc-modified-id="奇异值分解（SVD）-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>奇异值分解（SVD）</a></span></li><li><span><a href="#非负矩阵分解" data-toc-modified-id="非负矩阵分解-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>非负矩阵分解</a></span></li></ul></li><li><span><a href="#概率潜在语义分析(pLSA)" data-toc-modified-id="概率潜在语义分析(pLSA)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>概率潜在语义分析(pLSA)</a></span></li><li><span><a href="#潜在狄利克雷分配(LDA)" data-toc-modified-id="潜在狄利克雷分配(LDA)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>潜在狄利克雷分配(LDA)</a></span><ul class="toc-item"><li><span><a href="#Gibbs抽样" data-toc-modified-id="Gibbs抽样-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Gibbs抽样</a></span></li><li><span><a href="#随机梯度下降-(Stochastic-gradient-descent)" data-toc-modified-id="随机梯度下降-(Stochastic-gradient-descent)-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>随机梯度下降 (Stochastic gradient descent)</a></span><ul class="toc-item"><li><span><a href="#the-pLSA" data-toc-modified-id="the-pLSA-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>the pLSA</a></span></li><li><span><a href="#the-LDA" data-toc-modified-id="the-LDA-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>the LDA</a></span></li></ul></li></ul></li></ul></div>

# 潜在语义模型练习

主要参考赵航的书

# 潜在语义分析（LSA）

我们的文档包含两个主题：一个是粒子物理，常用词汇为"meson", "photon"。
另外一个是生活，常用词汇为"girl", "boy"。

In [1]:
import numpy as np

# documents set
documents=[
    "girl love boy",
    "boy love girl",
    "girl love girl",
    "boy love boy",
    
    "meson hits photon",
    "photon hits meson",
    "photon hits photon",
    "meson hits meson",

    "boy love meson",
    "girl love photon",
    "meson hits girl",
    "photon hits boy",
]

# setup terms set
terms_vec=[]
terms_set = set()
for doc in documents:
    terms = doc.split()
    for term in terms:
        if term not in terms_set:
            terms_vec.append(term)
            terms_set.add(term)
        
        
print("%d terms found in %d documents."%(len(terms_set), len(documents)))
print("terms:", terms_vec)


# setup terms index
terms_index = { }
for idx, term in enumerate(terms_set):
    terms_index[term] = idx


6 terms found in 12 documents.
terms: ['girl', 'love', 'boy', 'meson', 'hits', 'photon']


我们使用频率逆文档(TFIDF)来表示term-document矩阵。

In [2]:
# terms-documents matrix
tf_ij=np.zeros((len(terms_set),len(documents)))
tf_j=np.zeros(len(documents))  # number of terms in document j 
df_i=np.zeros(len(terms_set))  # number of documents which contain term i
df=len(documents)


term_doc_mat = np.zeros([len(terms_set), len(documents)])
for j, doc in enumerate(documents):
    terms = doc.split()
    tf_j[j] = len(terms)
    
    add_to_df_i=np.zeros(len(terms_set))
    for term in terms:
        i = terms_index[term]
        tf_ij[i][j] = tf_ij[i][j] + 1
        add_to_df_i[i] = 1
        
    df_i = df_i + add_to_df_i


# X in hang's book
term_doc_mat = tf_ij/tf_j*np.log(df/df_i).reshape(df_i.shape[0], 1)
        
with np.printoptions(precision=3):
    print("term-document (%dX%d) matrix setup"%(term_doc_mat.shape[0],term_doc_mat.shape[1]))
    print(term_doc_mat)

term-document (6X12) matrix setup
[[0.    0.    0.    0.    0.292 0.292 0.    0.584 0.292 0.    0.292 0.   ]
 [0.292 0.292 0.584 0.    0.    0.    0.    0.    0.    0.292 0.292 0.   ]
 [0.    0.    0.    0.    0.231 0.231 0.231 0.231 0.    0.    0.231 0.231]
 [0.292 0.292 0.    0.584 0.    0.    0.    0.    0.292 0.    0.    0.292]
 [0.    0.    0.    0.    0.292 0.292 0.584 0.    0.    0.292 0.    0.292]
 [0.231 0.231 0.231 0.231 0.    0.    0.    0.    0.231 0.231 0.    0.   ]]


## 奇异值分解（SVD）

将term-documents矩阵分解为 u s vT
保留其中主值最大的部分。
奇异值分解的正交性要求，矩阵的部分数值小于0，如何解释呢？

In [3]:
from numpy.linalg import svd

u,s,vT = svd(term_doc_mat)

#print("term-topic matrix:\n", u)
#print("topic-document matrix:\n", vT)
#print("main value:\n", s)

def print_mat(term_topic_mat, topic_document_mat):
    print("doc-topic matrix")
    print(" %20s   %10s  %10s"%("","topic0", "topic1"))
    for i, doc in enumerate(documents):
        print("`%20s'  %+10f  %+10f"%(doc, topic_document_mat[0][i], topic_document_mat[1][i]))

    print("term-topic matrix")
    print(" %20s   %10s  %10s"%("","topic0", "topic1"))

    for i, term in enumerate(terms_vec):
        print(" %20s   %+10f  %+10f"%(term, term_topic_mat[i][0], term_topic_mat[i][1]))

print_mat(u, vT)

doc-topic matrix
                            topic0      topic1
`       girl love boy'   -0.288675   -0.347652
`       boy love girl'   -0.288675   -0.347652
`      girl love girl'   -0.288675   -0.347652
`        boy love boy'   -0.288675   -0.347652
`   meson hits photon'   -0.288675   +0.347652
`   photon hits meson'   -0.288675   +0.347652
`  photon hits photon'   -0.288675   +0.347652
`    meson hits meson'   -0.288675   +0.347652
`      boy love meson'   -0.288675   -0.090976
`    girl love photon'   -0.288675   -0.090976
`     meson hits girl'   -0.288675   +0.090976
`     photon hits boy'   -0.288675   +0.090976
term-topic matrix
                            topic0      topic1
                 girl    -0.436281   +0.422454
                 love    -0.436281   -0.422454
                  boy    -0.345423   +0.378239
                meson    -0.436281   -0.422454
                 hits    -0.436281   +0.422454
               photon    -0.345423   -0.378239


## 非负矩阵分解

将term-documents矩阵$X$分解为 $WH$，其中$W$和$H$为（任意元素）非负矩阵。
目标为优化
$$\sum_{ij}(WH-X)_{ij}^2$$
优化方法采用“乘法更新规则”。

In [4]:
from numpy.linalg import norm

ntopics = 2
print("We want %d topics"%ntopics)

W = np.random.rand(len(terms_vec), ntopics)
H = np.random.rand(ntopics, len(documents))
# W * H -> X (term-document matrix)
for i in range(100):
    H1 = H * ((W.transpose().dot(term_doc_mat)) / (W.transpose().dot(W).dot(H)))
    dH = H1 - H
    H = H1
    W1 = W*((term_doc_mat.dot(H.transpose())) / (W.dot(H.dot(H.transpose()))))
    dW = W1 - W
    W = W1
    #print(np.power(dH,2).sum())
    #print(np.power(dW,2).sum())
    #print(np.power(term_doc_mat-W.dot(H),2).sum())

print_mat(W, H)

We want 2 topics
doc-topic matrix
                            topic0      topic1
`       girl love boy'   +0.000000   +0.215870
`       boy love girl'   +0.000000   +0.215870
`      girl love girl'   +0.000000   +0.215869
`        boy love boy'   +0.000000   +0.215870
`   meson hits photon'   +0.253443   +0.000000
`   photon hits meson'   +0.253443   +0.000000
`  photon hits photon'   +0.253443   +0.000000
`    meson hits meson'   +0.253443   +0.000000
`      boy love meson'   +0.092725   +0.134097
`    girl love photon'   +0.092728   +0.134095
`     meson hits girl'   +0.157436   +0.078981
`     photon hits boy'   +0.157434   +0.078980
term-topic matrix
                            topic0      topic1
                 girl    +1.135731   +0.024257
                 love    +0.020654   +1.333411
                  boy    +0.948347   +0.000000
                meson    +0.020633   +1.333420
                 hits    +1.135736   +0.024242
               photon    +0.000000   +1.113411


# 概率潜在语义分析(pLSA)

概率潜在语义分析即潜在语义分析概率化版本，是一种生成模型。

设文档集合为$D=\{d_j\}, j < |D|$，单词集合为$W=\{w_i\}, i < |W|$，文档$d_j$也是单词的序列，序列长度为$|d_j|$（重复单词多次计数。）

概率模型给出文档$d_j$中某一单词位置（比如第一个单词），单词$w_i$出现的概率$P(w_i, d_j)$为
$$
P(w_i, d_j) = 
\sum_k P(w_i, z_k) P(z_k, d_j)
$$
其中$P(z_k, d_j)$为文档$d_j$中某一单词选取话题$z_k$的概率，$P(w_i, z_k)$为话题$z_k$中单词$w_i$出现的概率。

设文档$d_j$第$i^\prime$个单词是单词$w_{i(d_j, i^\prime)}$（$i^\prime < |d_j|$），
则文档$d_j$生成的概率为
$$
\prod_{i^\prime < |d_j|} P(w_{i(d_j, i^\prime)}, d_j)
=\prod_{i < |W|} { P(w_i, d_j)^{n(w_i, d_j)} }
$$
$n(w_i, d_j)$为单词$i$在文档$j$中出现的频次。
所有文档的生成概率的对数为
$$
L = \sum_j \sum_{i} n(w_i, d_j) \log P(w_i, d_j)
$$
优化方法采用EM算法。


In [5]:
ntopics = 2
print("We want %d topics"%ntopics)


Pwz = np.random.rand(len(terms_vec), ntopics)
Pzd = np.random.rand(ntopics, len(documents))

nwd = np.zeros((len(terms_vec), len(documents))) # number of word `w` in document `d`

# setup terms set
for j, doc in enumerate(documents):
    terms = doc.split()
    for term in terms:
        i = terms_index[term]
        nwd[i][j] = nwd[i][j] + 1


for i in range(100):
    
    
    
    nd = nwd.sum(axis=0)
    
    # add 1E-6 to avoid 0
    # we need to inverse the elements of this matrix
    Pwd = Pwz.dot(Pzd) + np.full((len(terms_vec),len(documents)), 1E-6)
    
    Pwz_new = (nwd/Pwd).dot(Pzd.transpose()) * Pwz
    Pwz_new = Pwz_new / Pwz_new.sum(axis=0)
    
    
    Pzd_new = (nwd/Pwd).transpose().dot(Pwz).transpose() * Pzd
    Pzd_new = Pzd_new / nd
        
    #print(np.power(Pwz_new-Pwz,2).sum())
    #print(np.power(Pzd_new-Pzd,2).sum())
    #print(np.power(Pzd_new,2).sum())
    
    Pwz = Pwz_new
    Pzd = Pzd_new


print_mat(Pwz, Pzd)

We want 2 topics
doc-topic matrix
                            topic0      topic1
`       girl love boy'   +0.999997   +0.000000
`       boy love girl'   +0.999997   +0.000000
`      girl love girl'   +0.999997   +0.000000
`        boy love boy'   +0.999997   +0.000000
`   meson hits photon'   +0.000000   +0.999997
`   photon hits meson'   +0.000000   +0.999997
`  photon hits photon'   +0.000000   +0.999997
`    meson hits meson'   +0.000000   +0.999997
`      boy love meson'   +0.666664   +0.333330
`    girl love photon'   +0.666664   +0.333330
`     meson hits girl'   +0.333330   +0.666664
`     photon hits boy'   +0.333330   +0.666664
term-topic matrix
                            topic0      topic1
                 girl    +0.000000   +0.333333
                 love    +0.333333   +0.000000
                  boy    +0.000000   +0.333334
                meson    +0.333333   +0.000000
                 hits    +0.000000   +0.333333
               photon    +0.333334   +0.000000


# 潜在狄利克雷分配(LDA)

设文档集合为$D=\{d_m\}, m<|D|$，单词集合为$W=\{w_i\},i < |W|$，文档$d_j$也是单词的序列，序列长度为$|d_m|$（重复单词多次计数。）。
话题向量为$\mathbf{z}=\{z_m\},m < |D|$。
单词向量为$\mathbf{w} = \{w_{mn}\}, m < |D|, n< |d_m|$。
话题向量为$\mathbf{z} = \{z_{mn}\}, m < |D|, n< |d_m|$。
话题向量先验计数为$\mathbf{\alpha} = \{\alpha_{mk}\}, m < |D|, k < K$。
话题词向量先验计数为$\mathbf{\beta} = \{\beta_{ki}\}, k < K, i < |W|$。


$$
P(\mathbf{w}, \mathbf{z}, \mathbf{\theta},\mathbf{\phi}| \alpha, \beta) = 
\left[\prod_k \text{Dir} (\phi_k|\beta)\right]
\left[\prod_m \text{Dir}(\theta_m|\alpha)\right]
\left[
\prod_{mn} \text{Cat}(z_{mn}|\theta_m) 
\text{Cat}(w_{mn}|\phi_{z_{mn}})
\right]
$$

我们想要求 $P(\mathbf{z},\theta,\phi|\mathbf{w})$
$$P(\mathbf{z},\theta,\phi|\mathbf{w})
=
P(\theta,\phi|\mathbf{z},\mathbf{w})
P(\mathbf{z},\mathbf{w})
/P(\mathbf{w})
=
P(\theta,\phi|\mathbf{z},\mathbf{w})
P(\mathbf{z}|\mathbf{w})
$$
其中

$$
P(\mathbf{z},\mathbf{w}) = \prod_{k} \frac{B(n_k+\beta)}{B(\beta)} \prod_{m} \frac{B(n_m+\alpha)}{B(\alpha)}
$$

和

$$
P(\theta,\phi|\mathbf{z},\mathbf{w})
= \left[\prod_m \text{Dir}(\theta_m|n_m+\alpha)\right] \left[\prod_k \text{Dir}(\phi_k|n_k+\beta)\right]
$$




## Gibbs抽样

$P(\mathbf{z}|\mathbf{w})$可以通过Gibbs抽样方法得到。
注意根据狄利克雷分布，
$\theta_m$ 的期望为 $$(n_m + \alpha)/\sum_k (n_{mk}+\alpha_{mk})$$
$\phi_k$ 的期望为 $$(n_k + \beta)/\sum_{i} (n_{ki}+\beta_{ki})$$

In [6]:
ntopics = 2
print("We want %d topics"%ntopics)

alpha=1
beta=1

n_mk=np.zeros((len(documents), ntopics))
n_kv=np.zeros((ntopics, len(terms_vec)))
n_m=np.zeros(len(documents))
n_k=np.zeros(ntopics)

doc_lengths=np.zeros(len(documents), dtype=np.int)
for m, doc in enumerate(documents):
    doc_lengths[m] = len(doc.split())    

k_mn=np.zeros((len(documents), np.max(doc_lengths)),dtype=np.int)
v_mn=np.zeros((len(documents), np.max(doc_lengths)),dtype=np.int)

for m, doc in enumerate(documents):
    terms = doc.split()
    for n, term in enumerate(terms):
        v = terms_index[term]
        v_mn[m, n] = v


# intialization randomly
# assign each word a topic
def intialize_randomly():
    n_mk.fill(0)
    n_m.fill(0)
    n_kv.fill(0)
    n_k.fill(0)
    k_mn.fill(0)
    
    for m in range(len(documents)):
        for n in range(doc_lengths[m]):
            v = v_mn[m, n]
            k = np.random.randint(ntopics, dtype=np.int)        
            n_mk[m, k] += 1 
            n_m[m] += 1 
            n_kv[k,v] += 1 
            n_k[k] += 1
            k_mn[m, n] = k
        
# generate P(z|w) sample
def zw_Gibbs_topic_prob(m, v):
    n_kv_ = (n_kv[:,v] + beta) / (n_kv[:,:] + beta).sum(axis=1)
    n_mk_ = (n_mk[m,:] + alpha) / (n_mk[m,:] + alpha).sum()
    return n_kv_*n_mk_

# generate P(z|w) sample
def zw_MH_topic_prob(m, v, k):
    n_kv_ = (n_kv[k,v] + beta) / (n_kv[k,:] + beta).sum()
    n_mk_ = (n_mk[m,k] + alpha) / (n_mk[m,:] + alpha).sum()
    return n_kv_*n_mk_

def MH_step(topic_prob):
    
    m = np.random.randint(len(documents), dtype=np.int)
    n = np.random.randint(doc_lengths[m], dtype=np.int)

    v = v_mn[m, n]
    k = k_mn[m, n]          
    kp = np.random.randint(ntopics, dtype=np.int)
    
    k_prob = topic_prob(m, v, k)
    kp_prob = topic_prob(m, v, kp)
    
    acc = kp_prob/(k_prob+1E-10)
    u = np.random.rand()
    if u < acc:
        # accept      
        n_mk[m, k] -= 1
        #n_m[m] -= 1
        n_kv[k, v] -= 1
        n_k[k] -= 1

        k_mn[m, n] = kp
        n_mk[m, kp] += 1
        #n_m[m] += 1
        n_kv[kp, v] += 1
        n_k[kp] += 1
    
    
def Gibbs_step(sample_topic):

    for m in range(len(documents)):
        for n in range(doc_lengths[m]):
            v = v_mn[m, n]

            # current topic
            k = k_mn[m, n]            
            n_mk[m, k] -= 1
            #n_m[m] -= 1
            n_kv[k, v] -= 1
            n_k[k] -= 1
            
            topics_prob = sample_topic(m, v)
            acc = topics_prob.cumsum()/topics_prob.sum()
            p = np.random.rand()
            kp = np.searchsorted(acc, p, side="left")
                        
            k_mn[m, n] = kp
            n_mk[m, kp] += 1
            #n_m[m] += 1
            n_kv[kp, v] += 1
            n_k[kp] += 1


def print_matrix():
    print("doc-topic counting")
    print(n_mk)
    print("terms counting in each doc")
    print(n_m)
    print("document length")
    print(lengths)
    print("topic-words counting")
    print(n_kv)
    print("terms counting in each topic")
    print(n_k)
    
    
    
    
intialize_randomly()
            
# warnming
for _ in range(1000):
    Gibbs_step(zw_Gibbs_topic_prob)

    
iters=5
theta_mk_iters=np.zeros((iters, len(documents), ntopics))
phi_kv_iters=np.zeros((iters, ntopics, len(terms_vec)))
    

# average
for it in range(iters):
    #Gibbs_step(zw_sample_topic)
    MH_step(zw_MH_topic_prob)
    
    theta_mk = (n_mk + alpha)/(n_mk + alpha).sum(axis=1, keepdims=True)
    phi_kv = (n_kv + beta)/(n_kv + beta).sum(axis=1, keepdims=True)
    #print_mat(phi_kv.transpose(), theta_mk.transpose())

    theta_mk_iters[it,:,:] = theta_mk
    phi_kv_iters[it,:,:] = phi_kv
    
    
print_mat(phi_kv_iters.mean(axis=0).transpose(), theta_mk_iters.mean(axis=0).transpose())

We want 2 topics
doc-topic matrix
                            topic0      topic1
`       girl love boy'   +0.800000   +0.200000
`       boy love girl'   +0.800000   +0.200000
`      girl love girl'   +0.800000   +0.200000
`        boy love boy'   +0.800000   +0.200000
`   meson hits photon'   +0.200000   +0.800000
`   photon hits meson'   +0.400000   +0.600000
`  photon hits photon'   +0.400000   +0.600000
`    meson hits meson'   +0.200000   +0.800000
`      boy love meson'   +0.600000   +0.400000
`    girl love photon'   +0.600000   +0.400000
`     meson hits girl'   +0.200000   +0.800000
`     photon hits boy'   +0.200000   +0.800000
term-topic matrix
                            topic0      topic1
                 girl    +0.041667   +0.291667
                 love    +0.250000   +0.083333
                  boy    +0.041667   +0.291667
                meson    +0.250000   +0.083333
                 hits    +0.125000   +0.208333
               photon    +0.291667   +0.041667


## 随机梯度下降 (Stochastic gradient descent)

we want to maximize
$$
P(\theta, \phi|\mathbf{w}) = \sum_{z}P(\theta, \phi|\mathbf{z}, \mathbf{w}) P(\mathbf{z}|\mathbf{w})
$$

then we need
$$
\partial_\theta P(\theta, \phi|\mathbf{w})
=
\partial_\theta \sum_{z}P(\theta, \phi|\mathbf{z}, \mathbf{w}) P(\mathbf{z}|\mathbf{w})
=
\sum_{z} P(\mathbf{z}|\mathbf{w}) \partial_\theta P(\theta, \phi|\mathbf{z}, \mathbf{w})
=0
$$

similarly
$$
\sum_{z} P(\mathbf{z}|\mathbf{w}) \partial_\phi P(\theta, \phi|\mathbf{z}, \mathbf{w})
=0
$$


We can sample $z$ with probability of $P(\mathbf{z}|\mathbf{w})$, and move $\theta$ and $\phi$ along the direction of 
$-\partial_\theta P(\theta, \phi|\mathbf{z}, \mathbf{w})$ and $-\partial_\phi P(\theta, \phi, \mathbf{z}|\mathbf{w})$, respectively.
(At least) for small engouh the step, we will reach the statination point.
But it's often bad to use gradient decent method with probablity (we should use log(probablity)). 

we write it in the form of 
$$
P(\theta, \phi|\mathbf{w}) 
=
\sum_{z} P(\theta, \phi, \mathbf{z}, \mathbf{w})/P(\mathbf{w})
=
\sum_{z} P(\mathbf{z}, \mathbf{w} | \theta, \phi)P(\theta,\phi)/P(\mathbf{w})
$$

### the pLSA
EM algoirhtm, we define the Q function as
$$
\sum_{z} P(\mathbf{z} | \mathbf{w}, \theta^i, \phi^i) \log  P(\mathbf{z}, \mathbf{w}|\theta, \phi)
$$
We to maximize to Q (with constraint of $\sum_k \theta_{mk}=1$, or normalize in time)
$$
\partial_\theta \sum_{z}  P(\mathbf{z} | \mathbf{w}, \theta^i, \phi^i) \log  P(\mathbf{z}, \mathbf{w}|\theta, \phi) = 0
$$
$$
\sum_{z}  P(\mathbf{z} | \mathbf{w}, \theta^i, \phi^i) \partial_\theta \log  P(\mathbf{z}, \mathbf{w}|\theta, \phi) = 0
$$


We can sample $z$ with probability of $ P(\mathbf{z} | \mathbf{w}, \theta^i, \phi^i)$, and move $\theta$ and $\phi$ along the direction of 
$-\partial_\theta \log  P(\mathbf{z}, \mathbf{w}|\theta, \phi)$ and $-\partial_\phi \log  P(\mathbf{z}, \mathbf{w}|\theta, \phi)$, respectively.
The update ruler
$$
\theta^{i+1} = \theta + \eta \partial_\theta \log  P(\mathbf{z}, \mathbf{w}|\theta, \phi)
$$
$$
\phi^{i+1} = \phi + \eta \partial_\phi\log  P(\mathbf{z}, \mathbf{w}|\theta, \phi)
$$

It's simple that
$$
\log P(\mathbf{z},\mathbf{w}|\theta,\phi)
=
\prod_{mn} \text{Cat}(z_{mn}|\theta_m) 
\text{Cat}(w_{mn}|\phi_{z_{mn}})
$$
then
$$
\partial_{\theta_m} \log P(\mathbf{z},\mathbf{w}|\theta,\phi)
=
n_m/\theta_m
$$
$$
\partial_{\phi_k} \log P(\mathbf{z},\mathbf{w}|\theta,\phi)
=
n_k/\phi_k
$$

Excisie:
Note the normalized pdf of $\mathbf{z}$ and $\mathbf{w}$ should be
$$
P(\mathbf{z}, \mathbf{w}|\theta, \phi)
 =
P(\mathbf{z}, \mathbf{w}, \theta, \phi)/P(\theta, \phi)
 =
P(\theta, \phi | \mathbf{z}, \mathbf{w}) P(\mathbf{z}, \mathbf{w})/P(\theta, \phi)
$$

we just recall
$$
P(\theta, \phi)
=
\left[\prod_k \text{Dir} (\phi_k|\beta)\right]
\left[\prod_m \text{Dir}(\theta_m|\alpha)\right]
$$

$$
\log P(\mathbf{z},\mathbf{w}|\theta,\phi)
= 
\log P(\mathbf{z}, \mathbf{w})
+ \log \left[\prod_m \text{Dir}(\theta_m|n_m+\alpha)\right]
+\log\left[\prod_k \text{Dir}(\phi_k|n_k+\beta)\right]
-\log \left[\prod_m \text{Dir}(\theta_m|\alpha)\right]
-\log \left[\prod_k \text{Dir} (\phi_k|\beta)\right]
$$
we can again obtain the $P(\mathbf{z}, \mathbf{w}|\theta, \phi)$

and 
$$
P(\mathbf{z} |\mathbf{w}, \theta, \phi)
=
P(\mathbf{z}, \mathbf{w} | \theta, \phi)/ P(\mathbf{w}|\theta, \phi)
\propto
\theta_{mk} \phi_{kn}
$$
for the document $m$ and word $n$




### the LDA
Like EM algoirhtm, we define the Q function as
$$
\sum_{z} P(\mathbf{z}, \mathbf{w}, \theta^i, \phi^i) \log  P(\mathbf{z}, \mathbf{w}, \theta, \phi)
$$
We to maximize to Q (with constraint of $\sum_k \theta_{mk}=1$, or normalize in time)
$$
\partial_\theta \sum_{z}  P(\mathbf{z}, \mathbf{w}, \theta^i, \phi^i) \log  P(\mathbf{z}, \mathbf{w}, \theta, \phi) = 0
$$
$$
\sum_{z}  P(\mathbf{z}, \mathbf{w}, \theta^i, \phi^i) \partial_\theta \log  P(\mathbf{z}, \mathbf{w}, \theta, \phi) = 0
$$


We can sample $z$ with probability of $ P(\mathbf{z}, \mathbf{w}, \theta^i, \phi^i)$, and move $\theta$ and $\phi$ along the direction of 
$-\partial_\theta \log  P(\mathbf{z}, \mathbf{w}, \theta, \phi)$ and $-\partial_\phi \log  P(\mathbf{z}, \mathbf{w}, \theta, \phi)$, respectively.
The update ruler
$$
\theta^{i+1} = \theta + \eta \partial_\theta \log  P(\mathbf{z}, \mathbf{w}, \theta, \phi)
$$
$$
\phi^{i+1} = \phi + \eta \partial_\phi\log  P(\mathbf{z}, \mathbf{w}, \theta, \phi)
$$

It's simple that
$$
\partial_{\theta_m} \log P(\mathbf{z},\mathbf{w}, \theta,\phi)
=
(n_m + \alpha - 1)/\theta_m
$$
$$
\partial_{\phi_k} \log P(\mathbf{z},\mathbf{w}, \theta,\phi)
=
(n_k + \beta - 1)/\phi_k
$$

and 
$$
P(\mathbf{z}, \mathbf{w}, \theta, \phi)
\propto
\theta_{mk} \phi_{kn}
$$
for the document $m$ and word $n$


In [7]:
alpha=1.0
beta=1.0

theta_mk = np.random.rand(len(documents), ntopics)
theta_mk = theta_mk / theta_mk.sum(axis=1, keepdims=True)
phi_kv = np.random.rand(ntopics, len(terms_vec))
phi_kv = phi_kv / phi_kv.sum(axis=1, keepdims=True)

def zwtp_Gibbs_topic_prob(m, v):    
    return theta_mk[m,:] * phi_kv[:,v]

def zwtp_MH_topic_prob(m, v, k):
    return theta_mk[m, k] * phi_kv[k, v]

intialize_randomly()

# warnming
for _ in range(1000):
    Gibbs_step(zwtp_Gibbs_topic_prob)

def normalize(mat):
    return mat/mat.sum(axis=1, keepdims=True)
    
eta = 0.01
for _ in range(1000):
    MH_step(zwtp_MH_topic_prob)
    
    #print(n_mk[:,:] + alpha - 1)
    #print(theta_mk)
    d = (n_mk[:,:] + alpha - 1)/theta_mk[:,:]
    theta_mk = theta_mk + eta*d
    theta_mk = normalize(theta_mk)
    
    d = (n_kv[:,:] + beta - 1)/phi_kv[:,:]
    phi_kv = phi_kv + eta*d
    phi_kv = normalize(phi_kv)
    
    #print(theta_mk)
    #print(phi_kv)
    #print_mat(n_kv.transpose(), n_mk.transpose())
    #print(theta_mk.sum(axis=1))
    #print(phi_kv.sum(axis=1))
    #print(k_mn)
    #print(n_kv)
    #print(n_mk)
    
print_mat(phi_kv.transpose(), theta_mk.transpose())

doc-topic matrix
                            topic0      topic1
`       girl love boy'   +0.000000   +1.000000
`       boy love girl'   +0.000000   +1.000000
`      girl love girl'   +0.000000   +1.000000
`        boy love boy'   +0.000000   +1.000000
`   meson hits photon'   +1.000000   +0.000000
`   photon hits meson'   +1.000000   +0.000000
`  photon hits photon'   +1.000000   +0.000000
`    meson hits meson'   +1.000000   +0.000000
`      boy love meson'   +0.000000   +1.000000
`    girl love photon'   +0.414214   +0.585786
`     meson hits girl'   +1.000000   +0.000000
`     photon hits boy'   +0.585786   +0.414214
term-topic matrix
                            topic0      topic1
                 girl    +0.274868   +0.122925
                 love    +0.122925   +0.274868
                  boy    +0.301103   +0.000000
                meson    +0.000000   +0.301103
                 hits    +0.301103   +0.000000
               photon    +0.000000   +0.301103
